In [7]:
import openai
import json

def parent_annotate(folder):
    #the full folder has a list of folders with a name
    #for each folder in the parent folder
    #we need to open the "task" folder
    #open the brief.md file
    #send in as input the entire brief.md file as the user prompt to gpt and gemini
    #set the system prompt manually for both models based on the final decided prompt
    #store the output nested list into a json file 
    #keys will be folder name 
    #values will be gpt output and gemini output
    final_json = []
    return final_json

In [8]:
sys_prompt = """


You are a task classifier model. You will be provided with a task in the form of a brief. This brief will contain deliverables, what needs to be done, and what is provided. In the real world - this brief was provided to an actual human who completed the task. We are a team of researchers interested in knowing how this task should be classified. Below you will find a task classification taxonomy. 

Capabilities - 4
Skills - 13

Capability 1 - Technical Capabilities
Skills - 

Data and Information Processing - Gathering and extracting information from various digital sources
Digital Tool Operations - Effectively operating specialized digital applications 
Digital Content Creation - Creating written digital content for various purposes 
Programming and Development - Creating functional programming scripts and applications 

Capability 2 - Cognitive Skills 
Skills - 

Analytical Thinking - Identifying meaningful trends and relationships in data
Creative Thinking -Producing novel concepts and approaches
Critical Thinking -Validating accuracy and reliability of digital information

Capability 3 - Digital Communication
Skills - 
clear message writing -Creating clear written messages in digital formats
visual aid usage -Using images and visual elements to convey information
Multimedia -Using audio and video for effective messaging
audience calibration -Tailoring digital communication to specific recipients

Capability 4 = Domain Knowledge
Skills - 
Industry Specific Knowledge -Sector-specific digital needs and applications
Functional Area Knowledge - Specialized technical knowledge


You will be given a brief - then you will read the entire brief and return four python lists. Each one-hot encoded for the skills you think this job needs. So if job needs digital tool operation, programming and development, creative thinking, visual aid usage, audience calibration, functional area knowledge, you will return 

[[0,1,0,1], [0,1,0], [0,1,0,1], [0,1]]

Please only return the vector and nothing else. Do not start your sentences or outputs with "sure here's ..." simply return the vector and stop outputs. 





"""

In [9]:
from dotenv import load_dotenv
import os
from litellm import completion
import google.generativeai as genai
import os
from openai import OpenAI


load_dotenv()  # Load from .env file in the current dir

api_key = os.getenv("litellm_API_KEY")
#print(api_key)




BASE_URL = "https://litellm.ml-serving-internal.scale.com/v1"
api_key = api_key

client = OpenAI(
    api_key=api_key,
    base_url=BASE_URL,
)


INTERNAL_USER = "internal"
cost_tags = {
    "metadata": {
        "tags": ["project:agent-arena"]
    }
}


response = client.chat.completions.create(
    model="gemini/gemini-2.0-flash",
    messages=[
        {"role": "system", "content": "say hello"},
        {"role": "user", "content":"say what model you are"}
    ],
    user=INTERNAL_USER,
    extra_body=cost_tags,
)

print(response.choices[0].message.content)
#sanity check - should say trained by google

I am a large language model, trained by Google.



In [ ]:
import os
import json
import openai
import google.generativeai as genai




def parent_annotate(parent_folder_path: str) -> dict:
    """Iterate over sub‑directories inside *parent_folder_path*, read the
    `task/brief.md` file in each, and annotate it with both GPT‑4o and Gemini.
    Returns a dictionary keyed by sub‑folder name with the two model outputs."""

    results: dict[str, dict[str, str]] = {}
    system_prompt = sys_prompt  # <-- Paste your system prompt here

    if not os.path.isdir(parent_folder_path):
        raise FileNotFoundError(f"Parent folder not found: {parent_folder_path}")

    for folder in filter(lambda p: os.path.isdir(os.path.join(parent_folder_path, p)),
                         os.listdir(parent_folder_path)):
        brief_path = os.path.join(parent_folder_path, folder, "task", "brief.*")
        if not os.path.isfile(brief_path):
            continue

        with open(brief_path, "r", encoding="utf-8") as f:
            try:
                brief = f.read()
            except:
                brief = "No brief found - if you encounter this prompt simply enter a list of all zeroes len 13"

        # GPT‑4o
        try:
            gpt_resp = client.chat.completions.create(
    model="openai/o3",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content":brief}
    ],
    user=INTERNAL_USER,
    extra_body=cost_tags,
)
            gpt_output = gpt_resp.choices[0].message.content.strip()
        except Exception as e:
            gpt_output = "nothing"
            pass

        # Gemini
        try:
            #model = genai.GenerativeModel("gemini-2.5-pro")
            gemini_resp =client.chat.completions.create(
    model="gemini/gemini-2.0-flash",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content":brief}
    ],
    

    user=INTERNAL_USER,
    extra_body=cost_tags,
)
            gemini_output = gemini_resp.choices[0].message.content.strip()   
        except Exception as e:
            gemini_output = "nothing"
            pass

        results[folder] = {
            "gpt_output": gpt_output,
            "gemini_output": gemini_output,
        }
    

    with open("annotation_results_new.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    return results


if __name__ == "__main__":
    parent_dir = "/Users/ankit.aich/Documents/Code/accepted_data/Accepted"
    output = parent_annotate(parent_dir)
    print(output)


{'MG2': {'gpt_output': '[[1,1,1,0],[1,1,1],[0,1,0,1],[1,1]]', 'gemini_output': '[[1,1,0,0], [1,1,0], [0,0,0,0], [1,1]]'}, 'MG3': {'gpt_output': '[[1,1,1,0],[1,0,0],[1,1,0,0],[0,0]]', 'gemini_output': '[[1,0,1,0], [0,0,0], [1,0,0,0], [0,0]]'}, 'MG4': {'gpt_output': '[[0,1,1,0], [0,1,0], [0,0,1,1], [0,1]]', 'gemini_output': '[[0, 0, 0, 0], [0, 1, 0], [1, 0, 1, 0], [0, 0]]'}, 'KC13': {'gpt_output': '[[0,1,1,0],[0,1,0],[0,1,0,1],[0,1]]', 'gemini_output': '[[0, 0, 1, 0], [0, 1, 0], [1, 1, 0, 0], [0, 0]]'}, 'KC14': {'gpt_output': '[[0,1,1,0], [0,1,0], [0,1,0,0], [1,0]]', 'gemini_output': '[[0,0,0,0], [0,1,0], [0,0,0,0], [0,0]]'}, 'KC15': {'gpt_output': '[[0,1,1,0],[0,1,0],[0,1,0,1],[0,0]]', 'gemini_output': '[[0,0,0,0], [0,1,0], [1,1,0,0], [0,0]]'}, 'KC36': {'gpt_output': '[[0,1,1,0],[0,1,1],[1,0,0,1],[1,0]]', 'gemini_output': '```python\n[[1,0,1,0], [1,1,1], [1,0,0,1], [0,0]]\n```'}, 'AD39': {'gpt_output': '[[1,1,1,0],[1,1,1],[1,1,0,1],[1,1]]', 'gemini_output': '[[0,1,1,0], [1,0,1], [0,1,0,

In [13]:
import os
import json
import glob
from docx import Document
import openai
import google.generativeai as genai

# Define or import your system prompt, client, user, cost_tags as needed
# Example placeholder:




def parent_annotate(parent_folder_path: str) -> dict:
    """Iterate over sub-directories inside *parent_folder_path*, read any
    `brief.md`, `brief.txt`, or `brief.docx` file in each `task/` folder,
    and annotate it using GPT-4o and Gemini. Returns a dictionary keyed by
    sub-folder name with both model outputs."""

    results: dict[str, dict[str, str]] = {}

    if not os.path.isdir(parent_folder_path):
        raise FileNotFoundError(f"Parent folder not found: {parent_folder_path}")

    for folder in filter(lambda p: os.path.isdir(os.path.join(parent_folder_path, p)),
                         os.listdir(parent_folder_path)):
        task_path = os.path.join(parent_folder_path, folder, "task")
        brief_files = glob.glob(os.path.join(task_path, "brief.*"))
        brief = None

        for file_path in brief_files:
            ext = os.path.splitext(file_path)[1].lower()
            try:
                if ext in [".md", ".txt"]:
                    with open(file_path, "r", encoding="utf-8") as f:
                        brief = f.read()
                    break
                elif ext == ".docx":
                    doc = Document(file_path)
                    brief = "\n".join(p.text for p in doc.paragraphs)
                    break
            except Exception:
                continue

        if not brief:
            brief = "No brief found - if you encounter this prompt simply enter a list of all zeroes len 13"

        # GPT‑4o
        try:
            gpt_resp = client.chat.completions.create(
                model="openai/o3",
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": brief}
                ],
                user=INTERNAL_USER,
                extra_body=cost_tags,
            )
            gpt_output = gpt_resp.choices[0].message.content.strip()
        except Exception as e:
            gpt_output = "nothing"

        # Gemini
        try:
            gemini_resp = client.chat.completions.create(
                model="gemini/gemini-2.0-flash",
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": brief}
                ],
                user=INTERNAL_USER,
                extra_body=cost_tags,
            )
            gemini_output = gemini_resp.choices[0].message.content.strip()
        except Exception as e:
            gemini_output = "nothing"

        results[folder] = {
            "gpt_output": gpt_output,
            "gemini_output": gemini_output,
        }

    with open("annotation_results_new.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    return results


if __name__ == "__main__":
    parent_dir = "/Users/ankit.aich/Documents/Code/accepted_data/Accepted"
    output = parent_annotate(parent_dir)
    print(output)


{'SJ146': {'gpt_output': '[[0,0,0,0], [0,0,0], [0,0,0,0], [0,0]]', 'gemini_output': '[0,0,0,0,0,0,0,0,0,0,0,0,0]'}, 'SJ97': {'gpt_output': '[[0,1,1,0],[0,1,0],[0,1,0,1],[0,1]]', 'gemini_output': '[[0,1,0,0], [0,1,0], [1,1,0,0], [0,0]]'}, 'MG2': {'gpt_output': '[[1,1,1,0], [1,1,1], [1,1,0,1], [1,1]]', 'gemini_output': '[[1,1,0,0], [1,1,0], [0,1,0,0], [1,0]]'}, 'SJ99': {'gpt_output': '[[0,1,1,0],[0,1,0],[0,1,0,0],[1,1]]', 'gemini_output': '[[0,1,0,0], [0,0,1], [0,0,0,0], [0,0]]'}, 'MG50': {'gpt_output': '[[0,1,1,0],[0,1,0],[1,1,0,1],[0,1]]', 'gemini_output': '[[0,0,1,0], [0,1,0], [1,1,0,0], [0,0]]'}, 'SJ39': {'gpt_output': '[[0,0,0,0], [0,0,0], [0,0,0,0], [0,0]]', 'gemini_output': '[0,0,0,0],[0,0,0],[0,0,0,0],[0,0]'}, 'AD1': {'gpt_output': '[[0,1,1,0],[0,1,0],[1,1,0,1],[0,1]]', 'gemini_output': '[[0,0,0,0], [0,1,0], [1,1,0,0], [0,0]]'}, 'OL95': {'gpt_output': '[[0,1,1,0],[1,1,0],[0,1,0,0],[1,0]]', 'gemini_output': '[[1,0,0,0], [0,1,0], [0,0,0,0], [0,0]]'}, 'SJ98': {'gpt_output': '[[0,1,1